In [1]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [45]:
# If it's for the first time, authentication is required before running this code
# https://developers.google.com/earth-engine/guides/python_install-conda#windows_5
ee.Initialize()

In [46]:
# earth engine location of the subsets of the 10km grid created separately in the script "00_create_10_by_10_grid.R"
# 10km grid shapefile needs to be uploaded to assets in your GEE before running this line
grid = ee.FeatureCollection("projects/ee-akawano/assets/grid_10km")

# google drive associated with your GEE account
output_drive = "s5p_O3_L3_10km"
timezone = "utc"

In [47]:
# Sentinel-5P OFFL O3: Offline Ozone
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3
# 2018-09-08 to 2022-12-14
maiac = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_O3").select("O3_column_number_density")
maiac_proj = maiac.first().projection()
pixel_res = maiac_proj.nominalScale().getInfo()

In [48]:
# set start_date and end_date
start_date_1 = pd.to_datetime("2018-10-01", format="%Y/%m/%d")
end_date_1 = pd.to_datetime("2022-08-01", format="%Y/%m/%d")

# Initialize an empty list to store the date strings for the first list
start_date_list = []

# Loop through the dates for the first list, incrementing by 3 months each time
while start_date_1 < end_date_1:
    start_date_list.append(start_date_1.strftime("%Y-%m-%d"))
    
    # Increment the start date by 3 months
    start_date_1 += relativedelta(months=3)

# Start and end dates for the second list
start_date_2 = pd.to_datetime("2019-01-01", format="%Y/%m/%d")
end_date_2 = pd.to_datetime("2023-01-01", format="%Y/%m/%d")

# Initialize an empty list to store the date strings for the second list
end_date_list = []

# Loop through the dates for the second list, incrementing by 3 months each time
while start_date_2 < end_date_2:
    # Format the date as a string in the year-month-day format
    end_date_list.append(start_date_2.strftime("%Y-%m-%d"))
    
    # Increment the start date by 3 months
    start_date_2 += relativedelta(months=3)

# Print the lists of date strings
print(start_date_list)
print(end_date_list)

['2018-10-01', '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01', '2022-07-01']
['2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01']


In [49]:
# Calculate how many days between start and end days
def calculate_length(start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    length_date = (end_date - start_date).days
    return length_date

In [50]:
def create_image(start_date, length_date):
    images_list = ee.List([])
    for i in range(0, length_date):
        # timezone is default utc in ee.Date.fromYMD
        start_date = pd.to_datetime(start_date)
        im_date = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day).advance(i, 'day')
        im_date_end = im_date.advance(1, 'day')
        im = maiac.filterDate(im_date, im_date_end).reduce(ee.Reducer.mean())
        im = ee.Image(im).set("start_date", im_date.format("yMMdd"))
        images_list = images_list.add(im)
    return ee.ImageCollection.fromImages(images_list)

In [51]:
# For each day, calculate average for each cell grid
def process_image(start_date, end_date):
    length_date = calculate_length(start_date, end_date)
    maiac_images = create_image(start_date, length_date)
    maiac_images_reduced = maiac_images.map(lambda daily_im: (
        daily_im.reduceRegions(
            collection = grid,
            reducer = ee.Reducer.mean(),
            crs = maiac_proj,
            scale = pixel_res
        ).map(lambda f: f.set("start_date", daily_im.get("start_date")))))
    grid_maiac = ee.FeatureCollection(maiac_images_reduced).flatten()
    return grid_maiac

In [52]:
# make a list of properties to export 
export_properties = ["grid_id", "mean", "start_date"]
export_properties

['grid_id', 'mean', 'start_date']

In [23]:
# loop using start_date_list and end_date_list
for start_date, end_date in zip(start_date_list, end_date_list):
        grid_O3 = process_image(start_date, end_date)
        x = "O3_10km_grid"
        y = start_date
        t = "to"
        z = end_date
        output_name = "_".join([x, y, t, z])
        aod_task = ee.batch.Export.table.toDrive(
            collection = grid_O3,
            folder = output_drive,
            description = output_name,
            fileFormat = "CSV",
            selectors = export_properties)
        print(f"saving file as", {output_name})
        aod_task.start()

saving file as {'O3_10km_grid_2018-10-01_to_2019-01-01'}
saving file as {'O3_10km_grid_2019-01-01_to_2019-04-01'}
saving file as {'O3_10km_grid_2019-04-01_to_2019-07-01'}
saving file as {'O3_10km_grid_2019-07-01_to_2019-10-01'}
saving file as {'O3_10km_grid_2019-10-01_to_2020-01-01'}
saving file as {'O3_10km_grid_2020-01-01_to_2020-04-01'}
saving file as {'O3_10km_grid_2020-04-01_to_2020-07-01'}
saving file as {'O3_10km_grid_2020-07-01_to_2020-10-01'}
saving file as {'O3_10km_grid_2020-10-01_to_2021-01-01'}
saving file as {'O3_10km_grid_2021-01-01_to_2021-04-01'}
saving file as {'O3_10km_grid_2021-04-01_to_2021-07-01'}
saving file as {'O3_10km_grid_2021-07-01_to_2021-10-01'}
saving file as {'O3_10km_grid_2021-10-01_to_2022-01-01'}
saving file as {'O3_10km_grid_2022-01-01_to_2022-04-01'}
saving file as {'O3_10km_grid_2022-04-01_to_2022-07-01'}
saving file as {'O3_10km_grid_2022-07-01_to_2022-10-01'}


In [25]:
start_date = "2018-09-08"
end_date = "2018-10-01"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-08_to_2018-10-01'}


In [26]:
start_date = "2022-10-01"
end_date = "2022-11-01"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2022-10-01_to_2022-11-01'}


In [35]:
# Error: 'O3_10km_grid_2018-09-08_to_2018-10-01
start_date = "2018-09-08"
end_date = "2018-09-20"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-08_to_2018-09-20'}


In [36]:
# Error: 'O3_10km_grid_2018-09-08_to_2018-10-01
start_date = "2018-09-20"
end_date = "2018-10-01"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-20_to_2018-10-01'}


In [ ]:
# Error O3_10km_grid_2018-09-08_to_2018-09-20, O3_10km_grid_2018-09-20_to_2018-10-01

In [39]:
# Error: O3_10km_grid_2018-09-08_to_2018-09-20
start_date = "2018-09-08"
end_date = "2018-09-11"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-08_to_2018-09-11'}


In [40]:
# Error: O3_10km_grid_2018-09-08_to_2018-09-20
start_date = "2018-09-11"
end_date = "2018-09-14"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-11_to_2018-09-14'}


In [41]:
# Error: O3_10km_grid_2018-09-08_to_2018-09-20
start_date = "2018-09-14"
end_date = "2018-09-17"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-14_to_2018-09-17'}


In [42]:
# Error: O3_10km_grid_2018-09-08_to_2018-09-20
start_date = "2018-09-17"
end_date = "2018-09-20"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-17_to_2018-09-20'}


In [43]:
# O3_10km_grid_2018-09-20_to_2018-10-01
start_date = "2018-09-20"
end_date = "2018-09-27"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-20_to_2018-09-27'}


In [44]:
# O3_10km_grid_2018-09-20_to_2018-10-01
start_date = "2018-09-27"
end_date = "2018-10-01"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-27_to_2018-10-01'}


In [53]:
# O3_10km_grid_2018-09-08_to_2018-09-11
start_date = "2018-09-08"
end_date = "2018-09-10"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-08_to_2018-09-10'}


In [55]:
# O3_10km_grid_2018-09-08_to_2018-09-11
start_date = "2018-09-10"
end_date = "2018-09-12"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-10_to_2018-09-12'}


In [56]:
# Error O3_10km_grid_2018-09-10_to_2018-09-12, O3_10km_grid_2018-09-09_to_2018-09-11, O3_10km_grid_2018-09-08_to_2018-09-10

In [58]:
# O3_10km_grid_2018-09-11_to_2018-09-14
start_date = "2018-09-12"
end_date = "2018-09-14"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-12_to_2018-09-14'}


In [57]:
# O3_10km_grid_2018-09-11_to_2018-09-14
start_date = "2018-09-11"
end_date = "2018-09-13"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-11_to_2018-09-13'}


In [59]:
# Error O3_10km_grid_2018-09-11_to_2018-09-13, O3_10km_grid_2018-09-12_to_2018-09-14

In [62]:
# O3_10km_grid_2018-09-20_to_2018-09-27
start_date = "2018-09-20"
end_date = "2018-09-22"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-20_to_2018-09-22'}


In [63]:
# O3_10km_grid_2018-09-20_to_2018-09-27
start_date = "2018-09-22"
end_date = "2018-09-24"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-22_to_2018-09-24'}


In [64]:
# O3_10km_grid_2018-09-20_to_2018-09-27
start_date = "2018-09-24"
end_date = "2018-09-26"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-24_to_2018-09-26'}


In [65]:
# O3_10km_grid_2018-09-20_to_2018-09-27
start_date = "2018-09-25"
end_date = "2018-09-27"
grid_O3 = process_image(start_date, end_date)
x = "O3_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_O3,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'O3_10km_grid_2018-09-25_to_2018-09-27'}


In [ ]:
# if you want to cancel tasks:
# https://code.earthengine.google.com/tasks